## 为链增加记忆能力
***
- 注意：简单的链的记忆添加可以使用v0.2的方式，复杂的官方推荐使用langgraph
- 短时记忆：InMemoryHistory
- 长时记忆 RunnableWithMessageHistory

#### InMemoryHistory

In [1]:

from typing import List  # 导入List类型提示
from pydantic import BaseModel, Field  # 导入Pydantic的BaseModel和Field
from langchain_core.chat_history import BaseChatMessageHistory  # 导入聊天历史基类

from langchain_core.messages import BaseMessage, AIMessage  # 导入消息基类和AI消息类



class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """内存中实现的聊天消息历史记录。"""

    messages: List[BaseMessage] = Field(default_factory=list)  # 使用空列表作为默认值存储消息

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """添加一组消息到存储中"""
        self.messages.extend(messages)

    def clear(self) -> None:
        """清空所有消息"""
        self.messages = []

# 这里我们使用全局变量来存储聊天消息历史。
# 这样可以更容易地检查它以查看底层结果。
store = {}  # 创建空字典用于存储不同会话的历史记录

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    """根据会话ID获取历史记录，如果不存在则创建新的"""
    if session_id not in store:
        store[session_id] = InMemoryHistory()  # 为新会话创建新的历史记录对象
    return store[session_id]


# 获取会话ID为"1"的历史记录
history = get_by_session_id("1")
# 添加一条AI消息到历史记录
history.add_message(AIMessage(content="你好"))  # 修改为中文消息
# 打印存储的所有历史记录
print(store)  # 将输出包含会话"1"的历史记录，其中有一条"你好"的AI消息


{'1': InMemoryHistory(messages=[AIMessage(content='你好', additional_kwargs={}, response_metadata={})])}


在链中增加短时记忆

In [2]:
! pip install langchain-deepseek

In [3]:

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder  # 导入聊天提示模板和消息占位符
from langchain_core.runnables.history import RunnableWithMessageHistory  # 导入带历史记录的可运行组件
from langchain_deepseek import ChatDeepSeek
import os

llm = ChatDeepSeek(
    model="Pro/deepseek-ai/DeepSeek-V3",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    api_base=os.environ.get("DEEPSEEK_API_BASE"),
)

# 创建聊天提示模板，包含系统提示、历史记录和用户问题
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个擅长{ability}的助手"),  # 系统角色提示，使用ability变量定义助手专长
    MessagesPlaceholder(variable_name="history"),  # 放置历史消息的占位符
    ("human", "{question}"),  # 用户问题的占位符
])

# 将提示模板与DS模型连接成一个链
chain = prompt | llm

# 创建带有消息历史功能的可运行链
chain_with_history = RunnableWithMessageHistory(
    chain,  # 基础链
    # 使用上一个示例中定义的get_by_session_id函数获取历史记录
    get_by_session_id,
    input_messages_key="question",  # 指定输入消息的键名
    history_messages_key="history",  # 指定历史消息的键名
)

# 首次调用链，询问余弦的含义
print(chain_with_history.invoke(  # noqa: T201
    {"ability": "math", "question": "余弦函数是什么意思？"},  # 输入参数
    config={"configurable": {"session_id": "foo"}}  # 配置会话ID为"foo"
))

# 打印存储中的历史记录
# 此时应包含第一次对话的问题和回答
print(store)  

content='余弦函数（通常记作 \\(\\cos\\)）是三角函数的一种，用于描述直角三角形中一个锐角的邻边与斜边的比值，或者在单位圆中表示横坐标与半径的关系。以下是余弦函数的详细解释：\n\n### 1. **直角三角形中的定义**\n在直角三角形中，余弦值定义为：\n\\[\n\\cos \\theta = \\frac{\\text{邻边}}{\\text{斜边}}\n\\]\n其中：\n- \\(\\theta\\) 是一个锐角，\n- **邻边** 是该角相邻的直角边，\n- **斜边** 是直角三角形的斜边（最长边）。\n\n**例子**：若一个角 \\(\\theta\\) 的邻边长为 3，斜边为 5，则 \\(\\cos \\theta = \\frac{3}{5}\\)。\n\n---\n\n### 2. **单位圆中的定义**\n在直角坐标系中，以原点为中心、半径为 1 的单位圆上，余弦值等于角度 \\(\\theta\\) 的终边与圆交点的 **横坐标（x 坐标）**：\n\\[\n\\cos \\theta = x\n\\]\n- 当 \\(\\theta\\) 从 \\(0\\) 增加到 \\(2\\pi\\)（360°）时，余弦值从 1 递减到 -1，再回到 1，呈现周期性变化。\n\n---\n\n### 3. **余弦函数的性质**\n- **周期性**：余弦函数的周期为 \\(2\\pi\\)（或 360°），即 \\(\\cos(\\theta + 2\\pi) = \\cos \\theta\\)。\n- **取值范围**：值域为 \\([-1, 1]\\)。\n- **偶函数**：满足 \\(\\cos(-\\theta) = \\cos \\theta\\)，图像关于 y 轴对称。\n- **与正弦函数的关系**：\\(\\cos \\theta = \\sin\\left(\\theta + \\frac{\\pi}{2}\\right)\\)。\n\n---\n\n### 4. **图像特征**\n余弦函数的图像（波形图）特点：\n- **起点**：\\(\\cos 0 = 1\\)。\n- **零点**：在 \\(\\theta = \\frac{\\pi}{2}, \\frac{3\\pi}{2}, \\ldo

In [4]:

# 第二次调用链，询问余弦的反函数
# 由于使用相同的会话ID，模型可以参考前一次对话的上下文
print(chain_with_history.invoke(  
    {"ability": "math", "question": "它的反函数是什么？"},  # 输入参数
    config={"configurable": {"session_id": "foo"}}  # 使用相同的会话ID
))
# 再次打印存储中的历史记录
# 此时应包含两次对话的完整历史
print(store) 

content='余弦函数的反函数称为 **反余弦函数**（或 **arccosine**），记作 \\(\\arccos(x)\\) 或 \\(\\cos^{-1}(x)\\)。它的定义和性质如下：\n\n---\n\n### 1. **定义**\n反余弦函数是余弦函数在特定定义域上的逆映射，即：\n\\[\ny = \\arccos(x) \\quad \\Leftrightarrow \\quad x = \\cos(y)\n\\]\n其中：\n- **输入 \\(x\\)**：取值范围为 \\([-1, 1]\\)（因为余弦值域是 \\([-1, 1]\\)）。\n- **输出 \\(y\\)**：取值范围为 \\([0, \\pi]\\)（称为“主值分支”），单位为弧度。\n\n---\n\n### 2. **为什么限制定义域？**\n余弦函数 \\(\\cos \\theta\\) 本身是 **周期性** 和 **非单射** 的（例如 \\(\\cos 0 = \\cos 2\\pi = 1\\)）。为了使其可逆，必须限制定义域为 \\([0, \\pi]\\)，此时余弦函数是严格单调递减的，从而保证反函数存在且唯一。\n\n---\n\n### 3. **图像与性质**\n- **图像**：反余弦函数的图像是余弦函数在 \\([0, \\pi]\\) 上的镜像对称（关于直线 \\(y = x\\)），如下所示：\n  - 从点 \\((-1, \\pi)\\) 递减到 \\((1, 0)\\)。\n- **性质**：\n  - **定义域**：\\(x \\in [-1, 1]\\)。\n  - **值域**：\\(y \\in [0, \\pi]\\)（即 \\(0^\\circ\\) 到 \\(180^\\circ\\)）。\n  - **导数**：\\(\\frac{d}{dx} \\arccos(x) = -\\frac{1}{\\sqrt{1-x^2}}\\)（可通过隐函数求导证明）。\n\n---\n\n### 4. **计算示例**\n**问题**：求 \\(\\arccos\\left(-\\frac{1}{2}\\right)\\) 的值。  \n**解**：  \n寻找角度 \\(y\\) 使得 \\(\\cos(y) = -\\f

增加用户与对话ID，精准控制记忆

In [5]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.runnables import (
    ConfigurableFieldSpec,
)
import os

llm = ChatDeepSeek(
    model="Pro/deepseek-ai/DeepSeek-V3",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    api_base=os.environ.get("DEEPSEEK_API_BASE"),
)

store = {}  # 创建空字典用于存储不同用户和对话的历史记录

def get_session_history(
    user_id: str, conversation_id: str
) -> BaseChatMessageHistory:
    """
    根据用户ID和对话ID获取聊天历史记录
    如果不存在则创建新的历史记录对象
    
    参数:
        user_id: 用户的唯一标识符
        conversation_id: 对话的唯一标识符
    
    返回:
        对应的聊天历史记录对象
    """
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = InMemoryHistory()
    return store[(user_id, conversation_id)]

# 创建聊天提示模板，包含系统提示、历史记录和用户问题
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个擅长{ability}的助手"),  # 系统角色提示
    MessagesPlaceholder(variable_name="history"),  # 历史消息占位符
    ("human", "{question}"),  # 用户问题占位符
])

# 将提示模板与DS模型连接成一个链
chain = prompt | llm

# 创建带有消息历史功能的可运行链，支持用户ID和对话ID配置
with_message_history = RunnableWithMessageHistory(
    chain,  # 基础链
    get_session_history=get_session_history,  # 获取历史记录的函数
    input_messages_key="question",  # 输入消息的键名
    history_messages_key="history",  # 历史消息的键名
    history_factory_config=[  # 历史记录工厂配置
        ConfigurableFieldSpec(
            id="user_id",  # 配置字段ID
            annotation=str,  # 类型注解
            name="用户ID",  # 字段名称
            description="用户的唯一标识符",  # 字段描述
            default="",  # 默认值
            is_shared=True,  # 是否在多个调用间共享
        ),
        ConfigurableFieldSpec(
            id="conversation_id",  # 配置字段ID
            annotation=str,  # 类型注解
            name="对话ID",  # 字段名称
            description="对话的唯一标识符",  # 字段描述
            default="",  # 默认值
            is_shared=True,  # 是否在多个调用间共享
        ),
    ],
)

# 调用链，询问余弦的含义
# 指定用户ID为"123"，对话ID为"1"
with_message_history.invoke(
    {"ability": "数学", "question": "李白的老婆叫什么？"},  # 输入参数
    config={"configurable": {"user_id": "123", "conversation_id": "2"}}  # 配置参数
)

AIMessage(content='关于李白妻子的姓名，历史记载并不详尽，但根据有限的史料和后世研究，可以整理出以下信息：\n\n1. **许氏**  \n   李白在27岁左右（约727年）定居安陆（今湖北安陆），与当地名门许氏结婚。许氏的祖父许圉师是唐高宗时期的宰相。这段婚姻持续约十年，许氏育有一子一女（伯禽、平阳）。许氏去世后，李白离开安陆。\n\n2. **宗氏**  \n   约750年，李白在梁园（今河南开封）与武则天时期宰相宗楚客的孙女宗氏再婚。两人感情深厚，宗氏曾为李白的仕途奔走。安史之乱后，李白因卷入永王李璘案被流放，宗氏为其奔走求救。此后宗氏出家为道，李白晚年漂泊，两人未再团聚。\n\n3. **其他记载**  \n   有野史提及李白可能另有妻室（如“刘氏”），但缺乏可靠证据。郭沫若在《李白与杜甫》中推测李白可能有过多次婚姻，但具体姓名无考。\n\n**结论**：  \n李白两位有明确记载的妻子为 **许氏** 和 **宗氏**，均出身唐代仕宦家族。由于唐代女性名字多不传于史，她们的完整姓名已不可考。其他说法多为后世演绎，需谨慎对待。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 299, 'prompt_tokens': 14, 'total_tokens': 313, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Pro/deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '0195d2ab10b4061bf5076541aa9158a4', 'finish_reason': 'stop', 'logprobs': None}, id='run-f7dcce6c-5ad7-4621-8b2a-9039388cfde5-0', usage_metadata={'input_tokens': 14, 'output_tokens': 299, 'total_tokens': 313, 'input_token_detail

In [6]:
print(store)  # 输出存储的历史记录

{('123', '2'): InMemoryHistory(messages=[HumanMessage(content='李白的老婆叫什么？', additional_kwargs={}, response_metadata={}), AIMessage(content='关于李白妻子的姓名，历史记载并不详尽，但根据有限的史料和后世研究，可以整理出以下信息：\n\n1. **许氏**  \n   李白在27岁左右（约727年）定居安陆（今湖北安陆），与当地名门许氏结婚。许氏的祖父许圉师是唐高宗时期的宰相。这段婚姻持续约十年，许氏育有一子一女（伯禽、平阳）。许氏去世后，李白离开安陆。\n\n2. **宗氏**  \n   约750年，李白在梁园（今河南开封）与武则天时期宰相宗楚客的孙女宗氏再婚。两人感情深厚，宗氏曾为李白的仕途奔走。安史之乱后，李白因卷入永王李璘案被流放，宗氏为其奔走求救。此后宗氏出家为道，李白晚年漂泊，两人未再团聚。\n\n3. **其他记载**  \n   有野史提及李白可能另有妻室（如“刘氏”），但缺乏可靠证据。郭沫若在《李白与杜甫》中推测李白可能有过多次婚姻，但具体姓名无考。\n\n**结论**：  \n李白两位有明确记载的妻子为 **许氏** 和 **宗氏**，均出身唐代仕宦家族。由于唐代女性名字多不传于史，她们的完整姓名已不可考。其他说法多为后世演绎，需谨慎对待。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 299, 'prompt_tokens': 14, 'total_tokens': 313, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Pro/deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '0195d2ab10b4061bf5076541aa9158a4', 'finish_reason': 'stop', 'logprobs': None}, id='run-f7dcce6c-5ad7-4621-

### 使用Redis构建长期记忆
***
- 安装redis
- 运行redis服务
- 配置长期记忆

![](redis.png)

启动redis服务器：
注意，因为使用了Docker，所以这里代码略有不同

In [8]:
! pip install -qU langchain-redis langchain-openai redis

测试redis连接正常

In [17]:
import os

# Use the environment variable if set, otherwise default to localhost
REDIS_URL = "redis://redis:6379"
print(f"Connecting to Redis at: {REDIS_URL}")

Connecting to Redis at: redis://redis:6379


依赖包

In [10]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_redis import RedisChatMessageHistory
from langchain_deepseek import ChatDeepSeek
import os

使用ds

In [11]:
llm = ChatDeepSeek(
    model="Pro/deepseek-ai/DeepSeek-V3",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    api_base=os.environ.get("DEEPSEEK_API_BASE"),
)

最简单的使用

In [18]:
# 初始化 Redis 聊天消息历史记录
# 使用 Redis 存储聊天历史，需要提供会话 ID 和 Redis 连接 URL
history = RedisChatMessageHistory(session_id="user_123", redis_url=REDIS_URL)
#history.clear()  # 首先清空历史记录
# 向历史记录中添加消息
history.add_user_message("你好，AI助手！2222")  # 添加用户消息
history.add_ai_message("你好！我今天能为你提供什么帮助？222")  # 添加AI回复消息

# 检索并显示历史消息
print("聊天历史：")
for message in history.messages:
    # 打印每条消息的类型和内容
    print(f"{type(message).__name__}: {message.content}")

聊天历史：
HumanMessage: 你好，AI助手！2222
AIMessage: 你好！我今天能为你提供什么帮助？222
